## 导入包和加载数据

In [137]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np

In [119]:
train=pd.read_csv('../input/train_xy.csv')
test=pd.read_csv('../input/test_all.csv')
train.shape,test.shape

((15000, 160), (10000, 159))

In [120]:
train.head()

,cust_id,cust_group,y,x_1,x_2,x_3,x_4,x_5,x_6,x_7,...,x_148,x_149,x_150,x_151,x_152,x_153,x_154,x_155,x_156,x_157
0,110000,group_3,0,0.354167,0.604988,-99,-99,-99,-99,-99,...,1,1,1,1,1,1,1,1,3,-99
1,110001,group_3,0,0.125000,0.012058,-99,-99,-99,-99,-99,...,1,1,1,1,1,1,1,1,2,2
2,110002,group_3,0,0.333333,0.565979,0,0,0,0,0,...,1,1,2,1,1,1,1,1,2,2
3,110003,group_3,0,0.208333,0.316209,0,0,0,0,1,...,2,1,1,1,1,1,1,1,2,4
4,110004,group_3,0,0.208333,0.008061,-99,-99,-99,-99,-99,...,1,1,1,1,1,1,1,1,2,1


In [121]:
test.head()

,cust_id,cust_group,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,...,x_148,x_149,x_150,x_151,x_152,x_153,x_154,x_155,x_156,x_157
0,1,group_1,0.291667,0.555388,-99,-99,-99,-99,-99,-99,...,1,1,1,1,1,1,2,2,2,3
1,2,group_3,0.270833,0.770302,0,0,0,0,1,1,...,1,1,1,1,1,1,2,2,1,10
2,3,group_3,0.354167,0.440327,0,0,0,0,4,3,...,1,1,1,1,1,1,1,1,3,3
3,4,group_1,0.208333,0.476509,0,0,0,0,1,1,...,1,1,1,1,1,1,1,1,1,4
4,5,group_1,0.125000,0.955286,0,0,0,0,2,1,...,1,1,1,1,1,1,1,1,1,10


## 统计下类别标签

In [122]:

print(train['y'].value_counts())
print(train['y'].value_counts()[0]/len(train))

0    14309
1      691
Name: y, dtype: int64
0.9539333333333333


类别不均衡

## 统计每列缺失值（-99）

In [123]:
# for i in (train==-99).astype(int).sum(axis=0):
#     print(i)

In [124]:
# pandas DataFrame中去掉缺失值多于80%的列
# def drop_col(df, cutoff=0.5):
#     n = len(df)
#     for col_name in df.columns:
#         cnt = (df[col_name]==-99).astype(int).sum()
#         if (float(cnt) / n) > cutoff:
#             df.drop(col_name, axis=0, inplace=True)
# # drop_col(train,cutoff=0.8)
# (train['x_92']==-99).astype(int).sum()

In [125]:
# 去除异常值
n_row=len(train)
print(n_row)
for col in train.columns:
    cnt=(train[col]==-99).astype(int).sum()
    if (float(cnt) / n_row) > 0.45:
#         print(col,cnt,float(cnt) / n_row)
        train.drop([col], axis=1, inplace=True)
        test.drop([col],axis=1,inplace=True)
print(train.shape)
print(test.shape)

15000
(15000, 81)
(10000, 80)


一共舍去了160-121=39个特征

In [126]:
train.head()

,cust_id,cust_group,y,x_1,x_2,x_39,x_40,x_41,x_42,x_43,...,x_148,x_149,x_150,x_151,x_152,x_153,x_154,x_155,x_156,x_157
0,110000,group_3,0,0.354167,0.604988,6,9,10,7,9,...,1,1,1,1,1,1,1,1,3,-99
1,110001,group_3,0,0.125000,0.012058,7,8,7,7,3,...,1,1,1,1,1,1,1,1,2,2
2,110002,group_3,0,0.333333,0.565979,2,1,3,6,5,...,1,1,2,1,1,1,1,1,2,2
3,110003,group_3,0,0.208333,0.316209,2,2,3,7,9,...,2,1,1,1,1,1,1,1,2,4
4,110004,group_3,0,0.208333,0.008061,2,9,11,10,9,...,1,1,1,1,1,1,1,1,2,1


In [127]:
train=pd.get_dummies(train,prefix=['cust_group'])
test=pd.get_dummies(test,prefix=['cust_group'])

In [128]:
print(train.shape,test.shape)

(15000, 83) (10000, 82)


## 构建数据集

In [129]:
y=train['y']
X=train.drop(['cust_id','y'],axis=1,inplace=True)
X=train
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
print(X_train.shape,X_test.shape)

(12000, 81) (3000, 81)


In [151]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error,accuracy_score,f1_score,precision_score,recall_score,roc_auc_score
from math import sqrt

In [134]:
clf=LogisticRegression()
clf.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [161]:
pred=clf.predict_proba(X_test)
print(pred[:,1])
pred=np.argmax(pred,axis=1)
print(sqrt(mean_squared_error(pred,y_test)))
print(roc_auc_score(pred,y_test))
print(y_test)
print(pred)

[0.10366804 0.1490092  0.01526174 ... 0.05143363 0.00181019 0.00144106]
0.21291625896895083
0.7274849899933289
11499    0
6475     0
13167    0
862      0
5970     0
6706     0
3017     0
3781     0
3898     0
2250     0
542      0
4351     0
12807    0
6631     0
6705     0
6978     0
7399     0
11119    0
12665    0
5304     0
2312     0
7040     0
1889     1
8991     0
169      0
1795     0
3975     0
6820     0
7826     0
11809    0
        ..
10235    0
12397    0
9526     0
12078    0
8094     0
6018     0
10903    0
13066    0
3984     0
8157     0
3578     0
5608     0
10372    0
420      0
4448     0
7728     0
6428     0
1496     0
1047     0
11618    0
8720     0
856      0
14355    0
8400     0
8193     0
6216     0
3585     0
10023    0
14044    0
12965    0
Name: y, Length: 3000, dtype: int64
[0 0 0 ... 0 0 0]


In [155]:
eval_x=test.drop(['cust_id'],axis=1,inplace=False)

In [163]:
submit_pred=clf.predict_proba(eval_x)
submit_pred=submit_pred[:,1]
submit_pred

array([0.01051507, 0.00468262, 0.02302438, ..., 0.07927538, 0.00819492,
       0.00255311])

In [166]:
test['pred_prob']=submit_pred
test[['cust_id','pred_prob']].to_csv('submit.csv',index=False)